### 폰트

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121918 files and direc

In [ ]:
import matplotlib as mpl
import matplotlib.font_manager as fm

print ('버전: ', mpl.__version__)
print ('설치 위치: ', mpl.__file__)
print ('설정 위치: ', mpl.get_configdir())
print ('캐시 위치: ', mpl.get_cachedir())


print ('설정파일 위치: ', mpl.matplotlib_fname())


font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')

# ttf 폰트 전체갯수
print(len(font_list))

font_list

버전:  3.7.1
설치 위치:  /usr/local/lib/python3.10/dist-packages/matplotlib/__init__.py
설정 위치:  /root/.config/matplotlib
캐시 위치:  /root/.cache/matplotlib
설정파일 위치:  /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/matplotlibrc
29


['/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCodingBold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf',
 '/usr/share/fonts/tru

In [ ]:
# seabron으로 그래프 스타일 설정
import seaborn as sns

"""
아래 5개중 원하는 그래프 스타일 설정.
"""
# sns.set_style('whitegrid')
sns.set_style('darkgrid')
# sns.set_style('dark')
# sns.set_style('white')
# sns.set_style('ticks')

#----------------------------------------------------위 아래가 아예 다른내용이지만, seaborn설정후 폰트적용 셀을 매번 꼭 다시 실행해줘야하기에 셀을 붙여놨습니다.

# matplot으로 한글 font 설정
import matplotlib
import matplotlib.pyplot as plt

# 한글 폰트 사용시 글자를 선명하게
%config InlineBackend.figure_format = 'retina'

# '-' 음수 부호 보이게 설정
matplotlib.rc('axes', unicode_minus=False)

# 한글 폰트 설치
# plt.rc('font', family='NanumBarunGothic')
plt.rc('font', family='NanumGothic')

### 라이브러리 호출

In [1]:
# 연산 처리  패키지
import pandas as pd
import numpy as np

# 시각화 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# 통계분석 패키지
import statsmodels.api as sm

# 그래프 설정
%matplotlib inline

# 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, roc_curve, mean_squared_error, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import math
# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

## Model 함수 정의

#### 모델 함수

In [3]:
# 로지스틱 회귀 함수
def logistic_classifier(X, y):

    # 평가지표 / best param 저장 list
    acc = []
    prec = []
    recall = []
    auc = []

    acc_opt = []
    prec_opt = []
    recall_opt = []

    opt_thres = []

    # LogisticRegression 초기화
    model = LogisticRegression()

    for i in range(50):

        # y 범주 비율에 맞춰 train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

        # 데이터 Random Sampling
        X_train_rd = X_train.sample(n = 8000)
        y_train_rd = y_train[X_train_rd.index]

        # SMOTE로 X oversampling
        smt = SMOTE(sampling_strategy = 'auto')
        X_train_sm, y_train_sm = smt.fit_resample(X_train_rd, y_train_rd)

        # 모델 학습 / 예측
        model.fit(X_train_sm, y_train_sm)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:,1] # 양성 클래스에 대한 확률 추출

        # thr = 0.5일 때 평가지표 저장
        acc.append((accuracy_score(y_test, y_pred)))
        prec.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        auc.append(roc_auc_score(y_test, y_proba))

        # Optimal thr 저장
        fper, tper, thresholds = roc_curve(y_test, y_proba)
        optimal_idx = np.argmax(tper - fper)   # fpr, tpr 간 차이가 가장 클 때의 index 저장

        # Optimal thr일 때 평가지표 저장
        y_optpred = (y_proba > thresholds[optimal_idx]).astype(int)

        acc_opt.append(accuracy_score(y_test, y_optpred))
        prec_opt.append(precision_score(y_test, y_optpred))
        recall_opt.append(recall_score(y_test, y_optpred))
        opt_thres.append(thresholds[optimal_idx])

        # 횟수 출력
        if((i+1)%10 == 0) :
            print(f"{i+1}번째 완료")

    # 결과 DataFrame 생성
    results = pd.DataFrame({
        '0.5 acc' : acc,
        '0.5 prec': prec,
        '0.5 recall': recall,
        'opt acc' : acc_opt,
        'opt prec' : prec_opt,
        'opt recall' : recall_opt,
        'auc': auc,
        'opt threshold' : opt_thres
    })

    return results

## Logistic Regression 결과

In [4]:
# 코랩 mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# base preprocess 데이터로부터 y값 읽어오기
base = pd.read_csv("/content/drive/MyDrive/기계학습의이해/Dataset/base_process.csv", encoding = "euc-kr", engine='python')
yn_y = base['사고유무']

In [8]:
# Logistic Classifier model fit

total = pd.DataFrame()
count = 0
for a in range(1,4):
    for b in range(1, 4):
        for c in range(1, 4):
            for d in range(1, 4):
                for e in range(1, 4):
                    for f in range(1, 3):
                        for g in range(1, 3):

                            name = f"{a}{b}{c}{d}{e}{f}{g}"

                            # X 지정
                            yn_X = pd.read_csv(f"/content/drive/MyDrive/기계학습의이해/Dataset/DF/{name}.csv", encoding = 'euc-kr', engine='python')

                            # smote 계산을 위해 data type float로 변경
                            for col_name in yn_X.columns:
                                yn_X[col_name] = yn_X[col_name].astype(float)

                            # 모델 실행
                            logistic_rst = logistic_classifier(yn_X, yn_y)

                            # 1 : csv 하나씩 만드는 코드
                            logistic_rst.to_csv(f"/content/drive/MyDrive/기계학습의이해/Dataset/Logistic Regression/{name}_result.csv", index = False, encoding="euc-kr")

                            # 2 : csv 전체 합쳐서 만드는 코드
                            logistic_rst['data'] = name
                            total = pd.concat([total, logistic_rst])

                            # 진행상황
                            count +=1
                            if (count%10 == 0) :
                                print("========== ", count, "개, ", count/972, "% 완료 ===========")

total.to_csv("/content/drive/MyDrive/기계학습의이해/Dataset/Logistic Regression/LR_total_result.csv", index = False, encoding="euc-kr")

10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
==========  10 개,  0.015432098765432098 % 완료 ===========
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
==========  20 개,  0.030864197530864196 % 완료 ===========
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
10번째 완

In [17]:
total.to_csv("/content/drive/MyDrive/기계학습의이해/Dataset/Logistic Regression/LR_total_result.csv", index = False, encoding="euc-kr")